In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
file_path = '/FileStore/silver/nyc_taxi_data/'
df_gold = (spark.read.format('parquet') \
        .option('header', 'true')
        .option('mode', 'FAILFAST')
        .load(file_path))

Create column trip_duration: trip duration in minutes

In [0]:
df_gold = df_gold.withColumn(
    "trip_duration",
    date_format(from_unixtime(unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")), "HH:mm:ss")
)

New Total Surcharges Column: Adding up all the charges

In [0]:
df_gold = df_gold.withColumn(
    "total_surcharges",
    col("mta_tax") +  col("congestion_surcharge") + col("Airport_fee") + col('extra')
)

New fare per passenger column: average cost per passenger

In [0]:
df_gold = df_gold.withColumn(
    "fare_per_passenger",
    col("total_amount") / col("passenger_count")
)

Deleting non-useful columns

In [0]:
df_gold = df_gold.drop("VendorID", "Store_and_fwd_flag")

In [0]:
df_gold.write.format("parquet") \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .save("/FileStore/gold/nyc_taxi_data")